In [ ]:
import sys
# sys.path.append('../../../FinNLP/')  # https://github.com/AI4Finance-Foundation/FinNLP
sys.path.append('/FinGPT/fingpt/FinNLP')

In [10]:
import accelerate
import bitsandbytes

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast   # 4.30.2
from peft import PeftModel  # 0.4.0
import torch

from finnlp.benchmarks.fpb import test_fpb # 用于金融新闻情感分析的基准数据集，包含 "positive"（正面）、"negative"（负面）和 "neutral"（中性）三个标签。
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions #FiQA SA 数据集包含 17k 条来自微博头条和金融新闻的句子。这些标签根据 BloombergGPT 论文中的描述被改为 "positive"（正面）、"negative"（负面）和 "neutral"（中性）。
from finnlp.benchmarks.tfns import test_tfns # TFNS 数据集是一个英文数据集，包含了一个被标注的与金融相关的推文语料库。这个数据集被用于将金融相关的推文按情感进行分类。
# 数据集包含 11,932 个被标注为 "Bearish"（看跌，等同于 "negative"）、"Bullish"（看涨，等同于 "positive"）和 "Neutral"（中性）的文档。

from finnlp.benchmarks.nwgi import test_nwgi 
# NWGI 数据集的标签是由 ChatGPT 生成的。训练集有 16.2k 个样本，测试集有 4.05k 个样本。
# 该数据集包含 7 个分类标签："strong negative"（强烈负面）、"moderately negative"（中度负面）、"mildly negative"（轻微负面）、"neutral"（中性）、"mildly positive"（轻微正面）、"moderately positive"（中度正面）、"strong positive"（强烈正面）

/home/zwucn/anaconda3/envs/FinGPT/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/zwucn/anaconda3/envs/FinGPT/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


## Load Model (Pick one according the model from the following blocks)

#### 1. **FInGPT v3.1** based on ChatGLM2, runable on 1 * RTX 3090

In [3]:
# v3.1
base_model = "THUDM/chatglm2-6b" # 加载基础模型
peft_model = "oliverwang15/FinGPT_v31_ChatGLM2_Sentiment_Instruction_LoRA_FT"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModel.from_pretrained(base_model, trust_remote_code=True, load_in_8bit = True, device_map = "auto") # 加载了基础模型
model = PeftModel.from_pretrained(model, peft_model) # 将基础模型和微调模型结合起来，从而得到一个经过特定任务微调的模型。
model = model.eval()

pytorch_model.bin.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.83G [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00004-of-00007.bin:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

...

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/7.82M [00:00<?, ?B/s]

#### 2. **FinGPT v3.2** based on Llama2, runable on 1 * A 100 and also runable on 1 * RTX 3090 as long as `load_in_8bit = True` is set in Line 8, but the speed is slower

In [15]:
# v3.2
# base_model = "meta-llama/Llama-2-7b-chat-hf"  # Access needed
base_model = "daryl149/llama-2-7b-chat-hf"   
peft_model = "oliverwang15/FinGPT_v32_Llama2_Sentiment_Instruction_LoRA_FT"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0", load_in_8bit = True) # 加载了基础模型
model = PeftModel.from_pretrained(model, peft_model)
model = torch.compile(model)  # Please comment this line if your platform does not support torch.compile
model = model.eval()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 

## Test

In [6]:
batch_size = 4

In [16]:
# FPB
res = test_fpb(model, tokenizer, batch_size = batch_size)



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer: 


Total len: 1212. Batchsize: 4. Total steps: 304


  0%|          | 0/304 [01:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 

In [ ]:
# FiQA
res = test_fiqa(model, tokenizer, prompt_fun = add_instructions, batch_size = batch_size)

Generating train split:   0%|          | 0/961 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/102 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/150 [00:00<?, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: This $BBBY stock options trade would have more than doubled your money https://t.co/Oa0loiRIJL via @TheStreet
Answer: 


Total len: 275. Batchsize: 4. Total steps: 69


  0%|          | 0/69 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU  has a total capacity of 23.69 GiB of which 56.19 MiB is free. Including non-PyTorch memory, this process has 23.63 GiB memory in use. Of the allocated memory 23.16 GiB is allocated by PyTorch, and 157.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
# TFNS
res = test_tfns(model, tokenizer, batch_size = batch_size)

Found cached dataset csv (/xfs/home/tensor_zy/.cache/huggingface/datasets/zeroshot___csv/zeroshot--twitter-financial-news-sentiment-467590c9f24f65ad/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/2 [00:00<?, ?it/s]



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: $ALLY - Ally Financial pulls outlook https://t.co/G9Zdi1boy5
Answer: 


Total len: 2388. Batchsize: 8. Total steps: 299


100%|██████████| 299/299 [03:32<00:00,  1.41it/s]

Acc: 0.8936348408710217. F1 macro: 0.866430480618281. F1 micro: 0.8936348408710217. F1 weighted (BloombergGPT): 0.8937510370439953. 


In [9]:
# NWGI
res = test_nwgi(model, tokenizer, batch_size = batch_size)

Found cached dataset parquet (/xfs/home/tensor_zy/.cache/huggingface/datasets/oliverwang15___parquet/oliverwang15--news_with_gpt_instructions-ec641c48430028ee/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: In the latest trading session, Adobe Systems (ADBE) closed at $535.98, marking a +0.31% move from the previous day.
Answer: 


Total len: 4047. Batchsize: 8. Total steps: 506


100%|██████████| 506/506 [07:55<00:00,  1.06it/s]

Acc: 0.6360266864343959. F1 macro: 0.6443667929544722. F1 micro: 0.6360266864343959. F1 weighted (BloombergGPT): 0.6324602746076219. 
